In [13]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling3D, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import backend as K

2023-10-15 04:57:12.078370: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-15 04:57:12.126213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 04:57:12.899183: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [15]:
X_train.shape, y_train.shape

((1000000, 200, 3), (1000000, 15, 1))

In [16]:
#!g2.1
def quantile_loss_1(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.1 * error, (0.1 - 1) * error))
    return loss

In [ ]:
#!g2.1
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(200, 3, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_1, optimizer=opt, metrics=[RootMeanSquaredError()])

In [ ]:
#!g2.1
model.summary()

In [9]:
#!g2.1
model.fit(X_train, y_train, batch_size=128, epochs=25)

2023-10-15 04:09:15.705198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-10-15 04:09:16.589133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-15 04:09:16.813713: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55ecc132a2a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-15 04:09:16.813748: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-10-15 04:09:16.817923: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-15 04:09:16.942888: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

Epoch 1/25
782/782 [==============================] - 14s 9ms/step - loss: 0.0874 - root_mean_squared_error: 1.0320
Epoch 2/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0513 - root_mean_squared_error: 0.6378
Epoch 3/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0451 - root_mean_squared_error: 0.5588
Epoch 4/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0418 - root_mean_squared_error: 0.5229
Epoch 5/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0392 - root_mean_squared_error: 0.4962
Epoch 6/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0375 - root_mean_squared_error: 0.4783
Epoch 7/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0362 - root_mean_squared_error: 0.4628
Epoch 8/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0350 - root_mean_squared_error: 0.4484
Epoch 9/25
782/782 [==============================] - 7s 9ms/step - loss: 0.034

In [10]:
#!g2.1
pred = model.predict(X_test)

3125/3125 [==============================] - 4s 1ms/step


In [11]:
#!g2.1
np.save('test_pred_01', pred)

In [12]:
#!g2.1
model.save("model_01.h5")

In [ ]:
#!g2.1
